1. [Выгрузка данных](#start)
2. [Предобработка данных](#preprocessing)
3. [Количество книг, вышедших после 1 января 2000 года](#books)
4. [Количество обзоров и средняя оценка для каждой книги](#rating)  
5. [Издательство, которое выпустило наибольшее число книг толще 50 страниц (исключение из анализа брошюр)](#publisher)
6. [Автор с самой высокой средней оценкой книг (только книги с 50 и более оценками)](#authors)
7. [Среднее количество обзоров от пользователей, которые поставили больше 48 оценок](#48_rat)
8. [Таблица по году публикации](#year)
9. [Таблица со средней оценкой](#ccount_rating)
10. [Общий вывод](#done)


## Выгрузка данных <a id="start"></a> 

In [1]:
# импорт библиотек
import pandas as pd
from sqlalchemy import text, create_engine
from datetime import datetime, timedelta

In [2]:
# настройка для вывода содержимого ячеек
pd.options.display.max_colwidth = 130

In [3]:
# установка параметров
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохранение коннектора
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

con=engine.connect()

## Предобработка данных <a id="preprocessing"></a> 

In [4]:
# функция для предобработки данных
def data_preprocessing(data):
    """Эта функция для предобработки данных.

    Она приводит наименования столбцов датафрейма к правильному виду, 
    выводит датафрейм и основную информацию на экран, 
    выводит количество пропущенных значений для каждого столбца датафрейма,
    проверяет количество пропусков в процентах и подсчитывает количество явных дубликатов.
    """
    
    # приведение наименования столбцов датафрейма к правильному виду
    data.columns = [x.lower().replace(' ', '_') for x in data.columns]
    print('Вывод первых 5 строк датафрейма на экран')  
    display(data.head())
    print(' ')   
    print('Вывод основной информации о датафрейме с помощью метода info()') 
    display(data.info())
    print(' ')   
    print('Вывод количества пропущенных значений для каждого столбца датафрейма')
    display(data.isna().sum())
    print(' ')   
    print('Проверка количества пропусков в процентах')
    display(pd.DataFrame(round(data.isna().mean()*100,)).style.background_gradient('coolwarm'))
    print(' ')   
    print('Подсчёт количества явных дубликатов')
    display(data.duplicated().sum())

In [5]:
# данные books
books_dataset = '''
            SELECT * FROM books
        '''
books = pd.io.sql.read_sql(sql=text(books_dataset), con = con)

In [6]:
data_preprocessing(books)

Вывод первых 5 строк датафрейма на экран


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


 
Вывод основной информации о датафрейме с помощью метода info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

 
Вывод количества пропущенных значений для каждого столбца датафрейма


book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64

 
Проверка количества пропусков в процентах


,0
book_id,0.000000
author_id,0.000000
title,0.000000
num_pages,0.000000
publication_date,0.000000
publisher_id,0.000000


 
Подсчёт количества явных дубликатов


0

**Вывод**  
Явных дубликатов и пропусков нет. Тип данных в столбце publication_date необходимо поменять на тип даты.

In [7]:
# изменение типа данных в столбцах датафрейма, используя обработку ошибок при приобразовании
try:
    books['publication_date'] = pd.to_datetime(books['publication_date'])
except:
    print('publication_date - ошибка') 

In [8]:
# проверка преобразования
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   book_id           1000 non-null   int64         
 1   author_id         1000 non-null   int64         
 2   title             1000 non-null   object        
 3   num_pages         1000 non-null   int64         
 4   publication_date  1000 non-null   datetime64[ns]
 5   publisher_id      1000 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 47.0+ KB


**Вывод**  
Преобразование выполнено верно.

In [9]:
# данные authors
authors_dataset = '''
            SELECT * FROM authors
        '''
authors = pd.io.sql.read_sql(sql=text(authors_dataset), con = con)

In [10]:
data_preprocessing(authors)

Вывод первых 5 строк датафрейма на экран


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


 
Вывод основной информации о датафрейме с помощью метода info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


None

 
Вывод количества пропущенных значений для каждого столбца датафрейма


author_id    0
author       0
dtype: int64

 
Проверка количества пропусков в процентах


,0
author_id,0.000000
author,0.000000


 
Подсчёт количества явных дубликатов


0

**Вывод**  
Пропусков и явных дубликатов нет. Типы данных верны.

In [11]:
# данные publishers
publishers_dataset = '''
            SELECT * FROM publishers
        '''
publishers = pd.io.sql.read_sql(sql=text(publishers_dataset), con = con)

In [12]:
data_preprocessing(publishers)

Вывод первых 5 строк датафрейма на экран


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


 
Вывод основной информации о датафрейме с помощью метода info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


None

 
Вывод количества пропущенных значений для каждого столбца датафрейма


publisher_id    0
publisher       0
dtype: int64

 
Проверка количества пропусков в процентах


,0
publisher_id,0.000000
publisher,0.000000


 
Подсчёт количества явных дубликатов


0

**Вывод**  
Пропусков и явных дубликатов нет. Типы данных верны.

In [13]:
# данные ratings
ratings_dataset = '''
            SELECT * FROM ratings
        '''
ratings = pd.io.sql.read_sql(sql=text(ratings_dataset), con = con)

In [14]:
data_preprocessing(ratings)

Вывод первых 5 строк датафрейма на экран


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


 
Вывод основной информации о датафрейме с помощью метода info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

 
Вывод количества пропущенных значений для каждого столбца датафрейма


rating_id    0
book_id      0
username     0
rating       0
dtype: int64

 
Проверка количества пропусков в процентах


,0
rating_id,0.000000
book_id,0.000000
username,0.000000
rating,0.000000


 
Подсчёт количества явных дубликатов


0

**Вывод**  
Пропусков и явных дубликатов нет. Типы данных верны.

In [15]:
# данные reviews
reviews_dataset = '''
            SELECT * FROM reviews
        '''
reviews = pd.io.sql.read_sql(sql=text(reviews_dataset), con = con)

In [16]:
data_preprocessing(reviews)

Вывод первых 5 строк датафрейма на экран


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


 
Вывод основной информации о датафрейме с помощью метода info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

 
Вывод количества пропущенных значений для каждого столбца датафрейма


review_id    0
book_id      0
username     0
text         0
dtype: int64

 
Проверка количества пропусков в процентах


,0
review_id,0.000000
book_id,0.000000
username,0.000000
text,0.000000


 
Подсчёт количества явных дубликатов


0

**Вывод**  
Пропусков и явных дубликатов нет. Типы данных верны.

In [17]:
# функция для того, чтобы делать запросы
def request(goal):
    return pd.io.sql.read_sql(sql=text(goal), con = con)

## Количество книг, вышедших после 1 января 2000 года <a id="books"></a> 

In [18]:
task_1 = '''                
             SELECT COUNT(title) AS count_books
                
             FROM books
                
             WHERE publication_date > '2000-01-01';
         ''' 

request(task_1)

,count_books
0,819


**Вывод**  
После 1 января 2000 года вышло 819 книг.

## Количество обзоров и средняя оценка для каждой книги <a id="rating"></a> 

In [19]:
task_2 = '''                
             SELECT b.book_id AS book_id,
                    b.title AS book_title,
                    COUNT(DISTINCT rew.review_id) AS count_review,
                    AVG(rat.rating) AS avg_rating
                       
             FROM books AS b
             
             LEFT JOIN reviews AS rew ON rew.book_id = b.book_id
             LEFT JOIN ratings AS rat ON rat.book_id = b.book_id
                
             GROUP BY b.title, b.book_id
                
             ORDER BY avg_rating;   
         ''' 

request(task_2)

,book_id,book_title,count_review,avg_rating
0,303,Harvesting the Heart,2,1.50
1,371,Junky,2,2.00
2,316,His Excellency: George Washington,2,2.00
3,202,Drowning Ruth,3,2.00
4,915,The World Is Flat: A Brief History of the Twenty-first Century,3,2.25
...,...,...,...,...
995,421,Marvel 1602,2,5.00
996,418,March,2,5.00
997,169,Crucial Conversations: Tools for Talking When Stakes Are High,2,5.00
998,901,The Walking Dead Book One (The Walking Dead #1-12),2,5.00


In [20]:
task_2 = '''                
             SELECT b.book_id AS book_id,
                    b.title AS book_title,
                    COUNT(DISTINCT rew.review_id) AS count_review,
                    AVG(rat.rating) AS avg_rating
                       
             FROM books AS b
             
             LEFT JOIN reviews AS rew ON rew.book_id = b.book_id
             LEFT JOIN ratings AS rat ON rat.book_id = b.book_id
                
             GROUP BY b.title, b.book_id
                
             ORDER BY count_review;   
         ''' 

request(task_2)

,book_id,book_title,count_review,avg_rating
0,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
1,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
2,221,Essential Tales and Poems,0,4.000000
3,808,The Natural Way to Draw,0,3.000000
4,191,Disney's Beauty and the Beast (A Little Golden Book),0,4.000000
...,...,...,...,...
995,733,The Giver (The Giver #1),6,3.750000
996,854,The Road,6,3.772727
997,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
998,696,The Da Vinci Code (Robert Langdon #2),6,3.830508


**Вывод**  
Средний рейтинг книг варьируется от 1.5 до 5, а количество обзоров от 0 до 7.

In [21]:
# подсчет количества ревью
reviews['review_id'].nunique()

2793

In [22]:
task_2_1 = '''                
             SELECT COUNT(DISTINCT review_id) AS count_review
                       
             FROM reviews;
         ''' 

request(task_2_1)

,count_review
0,2793


## Издательство, которое выпустило наибольшее число книг толще 50 страниц (исключение из анализа брошюр) <a id="publisher"></a> 

In [23]:
task_3 = '''                
            SELECT p.publisher_id AS publisher_id, 
                   p.publisher AS name_publisher,
                   COUNT(b.book_id) AS count_books
                       
            FROM publishers AS p
                
            INNER JOIN books AS b ON b.publisher_id = p.publisher_id
                
            WHERE b.num_pages > 50
                
            GROUP BY p.publisher_id
                
            ORDER BY count_books DESC
                
            LIMIT 1;
         ''' 

request(task_3)

,publisher_id,name_publisher,count_books
0,212,Penguin Books,42


**Вывод**  
Наибольшее количество книг выпустило издательство Penguin Books — 42 книги. Данное издательство выпустило наибольшее количество книг толще 50 страниц.

## Автор с самой высокой средней оценкой книг (только книги с 50 и более оценками) <a id="authors"></a> 

In [24]:
task_4 = '''                
            SELECT a.author_id AS author_id, 
                   a.author AS name_author,
                   AVG(rat.rating) AS avg_rating,
                   COUNT(rat.rating_id) AS count_ratings
                       
            FROM authors AS a
                
            INNER JOIN books AS b ON b.author_id = a.author_id
            INNER JOIN ratings AS rat ON rat.book_id = b.book_id
            
            WHERE b.book_id IN (
                                 SELECT b.book_id
                              
                                 FROM books AS b
                              
                                 INNER JOIN ratings AS rat ON b.book_id = rat.book_id
                              
                                 GROUP BY b.book_id
                              
                                 HAVING COUNT(rat.rating_id) >= 50
                                )
            
            GROUP BY a.author_id
                    
            ORDER BY avg_rating DESC
                
            LIMIT 1;
         ''' 

request(task_4)

,author_id,name_author,avg_rating,count_ratings
0,236,J.K. Rowling/Mary GrandPré,4.287097,310


**Вывод**  
Автор с самой высокой средней оценкой книг — J.K. Rowling/Mary GrandPré. Самый высокий средний рейтинг — 4.287097.

## Среднее количество обзоров от пользователей, которые поставили больше 48 оценок <a id="48_rat"></a> 

In [25]:
task_5 = '''                
            WITH av AS (
                         SELECT rew.username,
                         
                         COUNT(rew.review_id) AS count_reviews
                         
                         FROM reviews AS rew
                        
                         WHERE username IN (
                                             SELECT username
                                             
                                             FROM ratings
                                             
                                             GROUP BY username
                                             
                                             HAVING COUNT(rating_id) > 48
                                            )
                        
                         GROUP BY username
                        )
                        
            SELECT AVG(av.count_reviews) AS avg_review
                       
            FROM av
         ''' 

request(task_5)

,avg_review
0,24.0


**Вывод**  
В среднем пользователи оставляют 24 обзора (из пользователей, которые поставили более 48 оценок)

## Таблица по году публикации <a id="year"></a> 

Таблица, которая содержит по году публикации:
* количество издательств;  
* выпущенных книг;  
* сколько всего тысяч страниц было в изданных книгах;  
* использовать года, в которых издано более 30 книг.  

In [26]:
task_6 = '''
             SELECT COUNT(DISTINCT publisher_id) AS count_publishers,
                    COUNT(book_id) AS count_books,
                    SUM(num_pages) / 1000 AS thousand_pages,
                    EXTRACT(YEAR FROM CAST(publication_date AS date)) AS publication_year
             
             FROM books
             
             GROUP BY publication_year
             
             HAVING COUNT(book_id) > 30
             
             ORDER BY publication_year;
         ''' 

request(task_6)

,count_publishers,count_books,thousand_pages,publication_year
0,26,41,15,1999.0
1,35,38,13,2000.0
2,41,60,21,2001.0
3,62,94,38,2002.0
4,65,105,41,2003.0
5,88,124,46,2004.0
6,89,139,55,2005.0
7,109,184,68,2006.0
8,38,50,18,2007.0


**Вывод**  
Публикация более 30 книг проходила с 1999 по 2007 года.  Наибольшее количество книг было издано в 2006 году, а наименьшее — 2000 году.  
Наибольшее количество издательств приходится на 2006 год.

## Таблица со средней оценкой  <a id="count_rating"></a> 

В одной таблице два числа:  
* средняя оценка тех книг, на которые написало отзывов более 3 человек;  
* средняя оценка остальных книг;  
* выводы какой рейтинг больше.


In [28]:
task_7 = '''
            (WITH av_up AS (
                         SELECT rat.book_id,
                         
                         AVG(rat.rating) AS count_rating
                         
                         FROM ratings AS rat
                        
                         WHERE book_id IN (
                                             SELECT book_id
                                             
                                             FROM reviews
                                             
                                             GROUP BY book_id
                                             
                                             HAVING COUNT(book_id) <= 3 
                                            )
                        
                         GROUP BY book_id
                        )
            
            
            
            
            SELECT AVG(av_up.count_rating) AS avg_rating
                       
            FROM av_up)           
            
            
            UNION 
            
            
            (WITH av AS (
                         SELECT rat.book_id,
                         
                         AVG(rat.rating) AS count_rating
                         
                         FROM ratings AS rat
                        
                         WHERE book_id IN (
                                             SELECT book_id
                                             
                                             FROM reviews
                                             
                                             GROUP BY book_id
                                             
                                             HAVING COUNT(book_id) > 3 
                                            )
                        
                         GROUP BY book_id
                        )
            
            
            
            
            SELECT AVG(av.count_rating) AS avg_rating
                       
            FROM av)           
         '''

request(task_7)

,avg_rating
0,3.890496
1,3.925045


**Вывод**  
Таким образом, средняя оценка книг, которые оценили более 3 человек чуть ниже (3.890496), чем средний рейтинг книг, которые оценили менее 3 человека и менее (3.925045).

## Общий вывод  <a id="done"></a> 

* в исследовании представлены 5 датасетов. 
* после 1 января 2000 года вышло 819 книг.  
* средний рейтинг книг варьируется от 1.5 до 5.  
* количество обзоров книг варьируется от 0 до 7.
* если рассматривать издательства, которые выпустили книги более 50 страниц, то можно сделать вывод, что наибольшее количество книг выпустило издательство Penguin Books (42 книги).  
* J.K. Rowling/Mary GrandPré является автором с самой высокой средней оценкой книг (4.287097).  
* если рассматривать пользователей, которые поставили более 48 оценок, то можно сделать вывод, что в среднем пользователи оставляют 24 обзора.   
* публикация более 30 книг в году проходила с 1999 по 2007 года.   
* издательство наибольшего количества книг приходится на 2006 год, а наименьшее на 2000.  
* наименьшее количество издательств приходится на тот же год, что и издательство наименьшего кольчества книг — 2000 год.  
* средняя оценка книг, которые оценили более 3 человек чуть ниже (3.890496), чем средний рейтинг книг, которые оценили менее 3 человека и менее (3.925045).
